# Depuracion datos Inventario Forestal Nacional

El presente notebook realiza la exploración inicial y depuración de datos del Inventario Forestal Nacional.

Se requieren los siguientes tablas enformato csv:

1. `detritos`: Contiene los mediciones realizadas en detritos de madera.

2. `ampie`: Mediciones de árboles muertos en pie.
    
3. `vegetacion`: Mediciones de árboles vivos. 

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

# Asignar nombres de archivos a variables
detritos = "../data/IFN/detritos.csv"
#ampie =  "../data/IFN/dasometricos/amp.csv"
#vegetacion = "../data/IFN/dasometricos/vegetacion.csv"

# Leer archivos como Pandas dataframes
det = pd.read_csv(detritos) 
#amp = pd.read_csv(ampie)
#veg = pd.read_csv(vegetacion)

In [ ]:
# Campos de mediciones de detritos

CONS = "CONS" # Indice de medicion (int64)
PLOT = "PLOT" # Indice conglomerado (int64)
TRAN = "TRAN" # Transecto de detritos (str: 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'). Hay registros sin transectos!!!
SECC = "SECC" # Seccion del transecto donde se registro la pieza (str: 'II', 'III', 'I'). Hay registros sin seccion!!!!
PIEZA = "PIEZA" # Numero consecutivo de la pieza en el transecto (float64). Deberia ser int64.
TIPO = "TIPO" # Tipo de detrito (str: 'DFM', 'DGM')
DIST = "DIST" # Distancia del detrito en cada seccion (float64 0.0-10.0)
AZIMUT = "AZIMUT" # Orientacion de la pieza respecto al transecto ?????????? (float64?, 0-360)
D1 = "D1" # Primer diametro de la pieza en cm (float64)
D2 = "D2" # Segundo diametro de la pieza en cm (float64)
INCL = "INCL" # Inclinacion de la pieza (float64). De acuerdo al manual deberia estar en el rango [-90, 90] pero en la hoja de calculo esta sesgada a valores positivos y hay valores [90, 180].
PI_cm = "PI_cm" # Penetracion del penetrometro en cm (float64). Verificar valores maximos.
PI_golpes = "PI_golpes" # Golpes ejecutados con el penetrometro (float64). Por que es un numero real????
PESO_RODAJA = "PESO_RODAJA" # Peso de la rodaja en gr (float64)
ESP1 = "ESP1" # Primer espesor de la rodaja en cm (float64)
ESP2 = "ESP2" # Segundo espesor de la rodaja en cm (float64)
ESP3 = "ESP3" # Tercer espesor de la rodaja en cm (float64)
ESP4 = "ESP4" # Cuarto espesor de la rodaja en cm (float64)
PESO_MUESTRA = "PESO_MUESTRA" # Peso fresco de la muestra en gr (float64)
VOL = "VOL" # Volumen de la muestra en ml (float64). Este campo no esta incluido en los formatos del INF!!!!
PESO_SECO = "PESO_SECO" # Peso fresco de la muestra en gr (float64)
DENS = "DENS" # Densidad de madera de la muestra en gr/ml (float64)

In [ ]:
# Convertir seccion muestreo de detritos en enteros para ahorrar espacio en db
try:
    det[SECC].replace(to_replace = ['I','II','III'], value = [1,2,3], inplace = True)
    # Si no existieran valores faltantes el reemplazo produciria una serie de int64, de lo contrario la columna resultante es float64
except:
    pass

for fi in [CONS, PLOT, PI_golpes, SECC]:
    if det[fi].dtype != np.int64:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, det[fi].dtype)
        if len(det[fi][det[fi].isna()]) > 1:
            print "Valores nulos son considerados np.float64:"
            print det[fi][det[fi].isna()]
        else:
            print "Valores np.float64 a revisar:"
            print det[fi][det[fi].map(lambda x: x % 1.0 != 0)].dropna()
            
        
for fi in [AZIMUT, PIEZA, DIST, D1, D2, INCL, PI_cm, PESO_RODAJA, ESP1, ESP2, ESP3, ESP4, PESO_MUESTRA, VOL, PESO_SECO, DENS]:
    if det[fi].dtype != np.float64:
        print "Campo {0} tiene tipo inapropiado ({1}en vez de float64).".format(fi, det[fi].dtype)

for fi in [TRAN, TIPO]:
    non_strings = det[fi].dropna()[~det[fi].dropna().apply(type).eq(str)]
    if len(non_strings):
        print "Campo {0} tiene tipo inapropiado ({1} en vez de str).".format(fi, non_strings.dtype)


In [ ]:
# Indice no debe contener duplicado
if len(det[det[CONS].duplicated()]):
    print "Tabla {0} contiene indices duplicados.".format(detritos)
    
for tr in det[TRAN].dropna().unique():
    if tr not in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
        print "`{0}` no es un valor valido de transecto de detrito".format(tr)
        
for tr in det[SECC].dropna().unique():
    if tr not in [1,2,3]:
        print "`{0}` no es un valor valido de transecto de detrito".format(tr)

for tr in det[TIPO].dropna().unique():
    if tr not in ['DFM', 'DGM']:
        print "`{0}` no es un valor valido de tipo de detrito".format(tr)
        
#
# Rangos Distancias
#
"""
if det[DIST].min() < 0:
    print "Rango distancia tiene valores negativos."
if det[DIST].max() > :
    print "Rango distancia sobrepasa el valor permitido."
"""

if det[AZIMUT].min() < 0:
    print "Rango Azimut tiene valores negativos."
if det[AZIMUT].max() > 360:
    print "Rango Azimut sobrepasa el valor permitido (360)"
    
#
# Rangos Diametros
#
"""
if det[D1].min() < or det[D2].min() <:
    print "Rango diametro tiene valores negativos."
if det[D1].max() > or det[D2].max() >:
    print "Rango diametro sobrepasa el valor permitido."
"""

if det[INCL].min() < -90:
    print "Rango inclinacion tiene valores menores al valor permitido (-90)."
if det[INCL].max() > 90:
    print "Rango inclinacion sobrepasa el valor permitido (90)"
    
if det[PI_cm].min() < 0:
    print "Hay valores negativos de entrada del penetrometro."
if det[PI_cm].max() > 25:
    print "Valores maximos del entrada del penetrometro so dudosos:"
    print det[[PI_cm, D1, D2]][det[PI_cm] > 25]

if det[PI_golpes].min() < 0:
    print "Valores negativos de golpes al penetrometro."
if det[PI_golpes].max() > 25:
    print "Valores maximos de golpes del penetrometro son dudosos:"
    print det[PI_golpes][det[PI_golpes] > 25]
